In [ ]:
import pandas as pd
import numpy as np
import math
import pandas as pd
from pyhdf.SD import SD, SDC
import os
import math
import datetime as dt
import h5py

In [ ]:
GRANULES_PATH="omino2/OMNO2dv003/" #path to OMNO2dv003 data
OUTPUT_PATH="data/" #path to store csv files
LOCATION_FILE = 'geonames-all-cities-with-a-population-1000-3.csv' #path to cities data 
WEEK_MAPPINGS = 'data/week_mappings_omino2.csv' #path to week mappings 
week_data = pd.read_csv(WEEK_MAPPINGS)
week_data=week_data.set_index("week")["granules"].to_dict()
for week in week_data:
    week_data[week]=week_data[week].split("#")

In [ ]:
metrics={}
metrics["ColumnAmountNO2"]="/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2"
metrics["ColumnAmountNO2CloudScreened"]="/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2CloudScreened"
metrics["ColumnAmountNO2Trop"]="/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2Trop"
metrics["ColumnAmountNO2TropCloudScreened"]="/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2TropCloudScreened"
#metrics["Weight"]="HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/Weight"
weight_path="HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/Weight"

In [ ]:
degree=0.25

In [ ]:
fp=open(LOCATION_FILE,"r")
evrythng=fp.readlines()
fp.close()
locations=set()
for line in evrythng[1:]:
    line=line.split("\n")[0].split(";")
    [lat,long]=line[-1].split(",")
    if degree==1:
        lat=min(math.ceil(float(lat)),90.0)-0.5
        long=min(math.ceil(float(long)),180.0)-0.5
    elif degree==0.05:
        lat=round(90-(int((90-float(lat))/0.05)*0.05+0.025),3)
        long=round(180-(int((180-float(long))/0.05)*0.05+0.025),3)
    elif degree==0.25:
        lat=round(90-(int((90-float(lat))/0.25)*0.25+0.125),3)
        long=round(180-(int((180-float(long))/0.25)*0.25+0.125),3)
    else:
        print("wrong degree resolution provided!")
    locations.add((lat,long))

In [ ]:
def get_indices(location):
    (lat,long)=location
    if degree==1:
        lat_start_index=int((90-lat-0.5)*4)
        lat_end_index=lat_start_index+4-1
        long_start_index=int((long-0.5+180)*4)
        long_end_index=long_start_index+4-1
    elif degree==0.25:
        lat_start_index=round(((90-lat-0.125)/0.25)*1)
        lat_end_index=lat_start_index+1-1
        long_start_index=round(((long-0.125+180)/0.25)*1)
        long_end_index=long_start_index+1-1
    lat_range=(lat_start_index,lat_end_index)
    long_range=(long_start_index,long_end_index)
    return [lat_range,long_range]

In [ ]:
OUTPUT_FILENAME=OUTPUT_PATH+"indian_cities_"+str(degree)+"degx"+str(degree)+"deg_omino2.csv"
fp=open(OUTPUT_FILENAME,"w")
fp.write(",".join(["location","week","metric","mean"])+"\n")
fp.close()
for week in week_data:
    print(week)
    week_metric_data={}
    for metric in metrics:
        week_metric_data[metric]={}
    for granule in week_data[week]:
        f=h5py.File(GRANULES_PATH+granule, "r")
        for metric in metrics:
            metric_data=np.array(f[metrics[metric]])
            weight_data=np.array(f[weight_path])
            for location in locations:
                indices=get_indices(location)
                [lat_range,long_range]=indices
                temp=metric_data[lat_range[0]:lat_range[1]+1,long_range[0]:long_range[1]+1]
                weights=weight_data[lat_range[0]:lat_range[1]+1,long_range[0]:long_range[1]+1]
                weights=weights[np.where(temp>-9999)]
                if len(weights)<=0:
                    continue
                temp=temp[np.where(temp>-9999)]
                if location not in week_metric_data[metric]:
                    week_metric_data[metric][location]=[[],[]]
                week_metric_data[metric][location][0]=np.append(week_metric_data[metric][location][0],temp)
                week_metric_data[metric][location][1]=np.append(week_metric_data[metric][location][1],weights)
    fp=open(OUTPUT_FILENAME,"a")
    for metric in week_metric_data:
        for location in week_metric_data[metric]:
            avg=np.average(week_metric_data[metric][location][0],weights=week_metric_data[metric][location][1])
            fp.write(",".join([str(location[0])+"#"+str(location[1]),week,metric]+[str(i) for i in [avg]])+"\n")
    fp.close()

1.2019.week1
1.2019.week2
1.2019.week3
1.2019.week4
2.2019.week1
2.2019.week2
2.2019.week3
2.2019.week4
3.2019.week1
3.2019.week2
3.2019.week3
3.2019.week4
4.2019.week1
4.2019.week2
4.2019.week3
4.2019.week4
5.2019.week1
5.2019.week2
5.2019.week3
5.2019.week4
6.2019.week1
6.2019.week2
6.2019.week3
6.2019.week4
7.2019.week1
7.2019.week2
7.2019.week3
7.2019.week4
8.2019.week1
8.2019.week2
8.2019.week3
8.2019.week4
9.2019.week1
9.2019.week2
9.2019.week3
9.2019.week4
10.2019.week1
10.2019.week2
10.2019.week3
10.2019.week4
11.2019.week1
11.2019.week2
11.2019.week3
11.2019.week4
12.2019.week1
12.2019.week2
12.2019.week3
12.2019.week4
1.2020.week1
1.2020.week2
1.2020.week3
1.2020.week4
2.2020.week1
2.2020.week2
2.2020.week3
2.2020.week4
3.2020.week1
3.2020.week2
3.2020.week3
3.2020.week4
4.2020.week1
4.2020.week2
4.2020.week3
4.2020.week4
5.2020.week1
5.2020.week2
5.2020.week3
5.2020.week4
6.2020.week1
6.2020.week2
6.2020.week3
6.2020.week4
7.2020.week1
7.2020.week2
7.2020.week3
7.2020.week4
